In [ ]:
!pip install torch --upgrade
!pip install -q --upgrade --no-cache-dir torch torchvision torchaudio

In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)

In [2]:
# from torchvision.ops.misc import ConvNormActivation
import torch
import torchvision
torchvision.__version__

'0.8.2'

In [8]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
# os.path.join("./work_dirs", '/*')
import glob
files = glob.glob("./work_dirs" + '/*')
for f in files:
  name = '/'+f.split('/')[-1]
  print(name)
  if name == '/mask_rcnn_swin-t-p4-w7_fpn_1x_coco':
    num = -1
    if not name[num].isdigit():
      name += '_1'
    while name[num-1].isdigit():
      num = -2
      t = int(name[-2:])+1
      name = name[:-2] + str(t)
  print(name)
cfg.evaluation 

/faster_rcnn_r50_fpn_1x_trash
/faster_rcnn_r50_fpn_1x_trash
/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_9
/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_9
/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_1
/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_1
/faster_rcnn_r50_fpn_1x_coco
/faster_rcnn_r50_fpn_1x_coco
/mask_rcnn_swin-t-p4-w7_fpn_1x_coco
/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_1
/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_10
/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_10


{'interval': 1, 'metric': 'bbox'}

In [4]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


In [5]:
import torch
print('torch version', torch.__version__)

torch version 1.7.1


In [6]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [7]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-03-30 02:07:05,870 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2022-03-30 02:07:05,871 - mmcv - INFO - load model from: torchvision://resnet50
2022-03-30 02:07:05,872 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2022-03-30 02:07:06,080 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2022-03-30 02:07:06,107 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-03-30 02:07:06,134 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-03-30 02:07:06,143 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [9]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)